# Procesamiento de Datos

## Agrupaciones en DataFrames

In [1]:
# Importar las librerias de trabajo
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from io import StringIO

In [2]:
# Generar el dataframe de trabajo
dframe = DataFrame({'k1':['X','X','Y','Y','Z'],
                    'k2':['alpha','beta','alpha','beta','alpha'],
                    'datos1':np.random.randn(5),
                    'datos2':np.random.randn(5)})
dframe

k1     k2    datos1    datos2
0  X  alpha -0.027194 -0.303951
1  X   beta  1.038000 -1.491602
2  Y  alpha  1.779571 -2.038314
3  Y   beta  0.227072 -1.407753
4  Z  alpha  1.327176  0.229930

In [3]:
# Ver como se usa la función "groupby"
grupo1 = dframe['datos1'].groupby(dframe['k1'])
grupo1

In [4]:
# Realizar operaciones sobre este grupo
grupo1.mean()

k1
X    0.505403
Y    1.003322
Z    1.327176
Name: datos1, dtype: float64

In [5]:
# Podemos pasar el nombre de la columna como llave de agrupación
dframe.groupby('k1').mean()

datos1    datos2
k1                    
X   0.505403 -0.897776
Y   1.003322 -1.723033
Z   1.327176  0.229930

In [6]:
# o multiples columnas
dframe.groupby(['k1','k2']).mean()

datos1    datos2
k1 k2                       
X  alpha -0.027194 -0.303951
   beta   1.038000 -1.491602
Y  alpha  1.779571 -2.038314
   beta   0.227072 -1.407753
Z  alpha  1.327176  0.229930

In [7]:
# Podemos utilizar groupby para contar elementos
dframe.groupby(['k1']).size()

k1
X    2
Y    2
Z    1
dtype: int64

In [8]:
# Podemos iterar sobre los grupos
# por ejemplo:
for nombre, grupo in dframe.groupby('k1'):
    print ("Este es el grupo %s" %nombre)
    print (grupo)
    print ('\n')

Este es el grupo X
  k1     k2    datos1    datos2
0  X  alpha -0.027194 -0.303951
1  X   beta  1.038000 -1.491602


Este es el grupo Y
  k1     k2    datos1    datos2
2  Y  alpha  1.779571 -2.038314
3  Y   beta  0.227072 -1.407753


Este es el grupo Z
  k1     k2    datos1   datos2
4  Z  alpha  1.327176  0.22993




In [9]:
# Podemos iterar sobre varias llaves
for (k1,k2), grupo in dframe.groupby(['k1','k2']):
    print ("Llave1 = %s Llave2 = %s" %(k1,k2))
    print (grupo)
    print ('\n')

Llave1 = X Llave2 = alpha
  k1     k2    datos1    datos2
0  X  alpha -0.027194 -0.303951


Llave1 = X Llave2 = beta
  k1    k2  datos1    datos2
1  X  beta   1.038 -1.491602


Llave1 = Y Llave2 = alpha
  k1     k2    datos1    datos2
2  Y  alpha  1.779571 -2.038314


Llave1 = Y Llave2 = beta
  k1    k2    datos1    datos2
3  Y  beta  0.227072 -1.407753


Llave1 = Z Llave2 = alpha
  k1     k2    datos1   datos2
4  Z  alpha  1.327176  0.22993




In [10]:
# Una táctica usada es crear un diccionario de los datos
grupo_d = dict(list(dframe.groupby('k1')))
grupo_d

{'X':   k1     k2    datos1    datos2
 0  X  alpha -0.027194 -0.303951
 1  X   beta  1.038000 -1.491602,
 'Y':   k1     k2    datos1    datos2
 2  Y  alpha  1.779571 -2.038314
 3  Y   beta  0.227072 -1.407753,
 'Z':   k1     k2    datos1   datos2
 4  Z  alpha  1.327176  0.22993}

In [11]:
grupo_d['Y']

k1     k2    datos1    datos2
2  Y  alpha  1.779571 -2.038314
3  Y   beta  0.227072 -1.407753

In [12]:
# Por ejemplo, si solo necesitamos agrupar la columna dato2 por ambos conjuntos de llaves
grupo_dato2 = dframe.groupby(['k1','k2'])[['datos2']]
grupo_dato2.mean()

datos2
k1 k2             
X  alpha -0.303951
   beta  -1.491602
Y  alpha -2.038314
   beta  -1.407753
Z  alpha  0.229930

## Agrupar series y diccionarios

In [13]:
# Aprendamos a usar diccionarios o series con groupby
# Hagamos un dataframe con diferentes animales
animales = DataFrame(np.arange(16).reshape(4, 4),
                   columns=['W', 'X', 'Y', 'Z'],
                   index=['Perro', 'Gato', 'Pajaro', 'Raton'])
print(animales)

         W   X   Y   Z
Perro    0   1   2   3
Gato     4   5   6   7
Pajaro   8   9  10  11
Raton   12  13  14  15


In [14]:
# Agreguemos algunos datos NAN
# animales.ix[1:2, ['W', 'Y']] = np.nan
animales.loc[1:2, ['W', 'Y']]  = np.nan
print(animales)

           W   X     Y   Z
Perro    0.0   1   2.0   3
Gato     NaN   5   NaN   7
Pajaro   8.0   9  10.0  11
Raton   12.0  13  14.0  15


In [15]:
# Ahora supongamos que tenemos un diccionario con valores de comportamiento
comportamiento_m = {'W': 'bien', 'X': 'mal', 'Y': 'bien','Z': 'mal'}

# Ahora podemos agrupar a los animales usando este mapeo
animales_col = animales.groupby(comportamiento_m, axis=1)

# Mostremos la suma, de acuerdo a la agrupación por el mapeo
animales_col.sum()

# por ejemplo [Perro][bien] = [Perro][Y]+[Perro][W]

bien   mal
Perro    2.0   4.0
Gato     0.0  12.0
Pajaro  18.0  20.0
Raton   26.0  28.0

In [16]:
# Ahora intentemos lo mismo con una serie
comportamiento_s = Series(comportamiento_m)
comportamiento_s

W    bien
X     mal
Y    bien
Z     mal
dtype: object

In [17]:
# Agrupemos la serie
animales.groupby(comportamiento_s, axis=1).count()

bien  mal
Perro      2    2
Gato       0    2
Pajaro     2    2
Raton      2    2

In [18]:
print(animales)

           W   X     Y   Z
Perro    0.0   1   2.0   3
Gato     NaN   5   NaN   7
Pajaro   8.0   9  10.0  11
Raton   12.0  13  14.0  15


In [19]:
# Supongamos que queremos agrupar por la longitud de los nombres de los animales, ¡podemos pasar la función len a groupby!
animales.groupby(len).sum()

# Nota: el índice ahora es el número de letras en el nombre del animal

W   X     Y   Z
4   0.0   5   0.0   7
5  12.0  14  16.0  18
6   8.0   9  10.0  11

In [20]:
# También podemos mezclar funciones con matrices, diccionarios y series para métodos groupby
# Se define una lista de llaves
llaves = ['A', 'B', 'A', 'B']

# Se agrupa la longitud del nombre y las llaves para mostrar los valores máximos
animales.groupby ([len, llaves]). max ()

W   X     Y   Z
4 B   NaN   5   NaN   7
5 A   0.0   1   2.0   3
  B  12.0  13  14.0  15
6 A   8.0   9  10.0  11

In [22]:
# También podemos usar groupby con niveles de índice jerárquico

# Crear un índice de columna jerárquica
jerarq_col = pd.MultiIndex.from_arrays([['MX','MX','MX','US','US'],[1,2,3,1,2]],names=['Ciudad','Valor'])

# Crear un dataframe con índice jerárquico
dframe = DataFrame(np.arange(25).reshape(5,5),columns=jerarq_col)

# Valores múltiples por 100 para mayor claridad
dframe = dframe*100
dframe

Ciudad    MX                US      
Valor      1     2     3     1     2
0          0   100   200   300   400
1        500   600   700   800   900
2       1000  1100  1200  1300  1400
3       1500  1600  1700  1800  1900
4       2000  2100  2200  2300  2400

## Agregación

In [24]:
# La agregación de datos consiste en operaciones que resultan en un escalar (por ejemplo, mean(), sum(), count(), etc.)
# Se guarda el archivo winquality.csv en la misma carpeta que los notebooks Python, tenga en cuenta el delimitador utilizado ;
# El archivo contiene medidas de características de diferentes vinos, como acidez, azúcar, densidad, etc.
# La columna de calidad (quality) es una calificación asignada a cada vino
dframe_vino = pd.read_csv('data/winequality-red.csv',sep=';')
dframe_vino

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
1594            6.2             0.600         0.08             2.0      0.090   
1595            5.9             0.550         0.10             2.2      0.062   
1596            6.3             0.510         0.13             2.3      0.076   
1597            5.9             0.645         0.12             2.0      0.075   
1598            6.0             0.310         0.47             3.6      0.067   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                    11.0                  34.0  0.99780  3.51       0.56   
1                    25.0                  67.0  0.99680  3.20       0.68   
2                    15.0                  54.0  0.99700  3.26       0.65   
3                    17.0                  60.0  0.99800  3.16       0.58   
4                    11.0                  34.0  0.99780  3.51       0.56   
...                   ...                   ...      ...   ...        ...   
1594                 32.0                  44.0  0.99490  3.45       0.58   
1595                 39.0                  51.0  0.99512  3.52       0.76   
1596                 29.0                  40.0  0.99574  3.42       0.75   
1597                 32.0                  44.0  0.99547  3.57       0.71   
1598                 18.0                  42.0  0.99549  3.39       0.66   

      alcohol  quality  
0         9.4        5  
1         9.8        5  
2         9.8        5  
3         9.8        6  
4         9.4        5  
...       ...      ...  
1594     10.5        5  
1595     11.2        6  
1596     11.0        6  
1597     10.2        5  
1598     11.0        6  

[1599 rows x 12 columns]

In [25]:
# ¿Qué tal si descubrimos el contenido promedio de alcohol para el vino?
dframe_vino['alcohol'].mean()

10.422983114446529

In [26]:
# Ese fue un ejemplo de agregación, ¿qué tal si hacemos el nuestro mediante una función?
def max_a_min(matriz):
    return matriz.max() - matriz.min()

# Agrupemos los vinos por su calidad (quality)
vino = dframe_vino.groupby('quality')
vino.describe()

fixed acidity                                                     \
                count      mean       std  min   25%   50%     75%   max   
quality                                                                    
3                10.0  8.360000  1.770875  6.7  7.15  7.50   9.875  11.6   
4                53.0  7.779245  1.626624  4.6  6.80  7.50   8.400  12.5   
5               681.0  8.167254  1.563988  5.0  7.10  7.80   8.900  15.9   
6               638.0  8.347179  1.797849  4.7  7.00  7.90   9.400  14.3   
7               199.0  8.872362  1.992483  4.9  7.40  8.80  10.100  15.6   
8                18.0  8.566667  2.119656  5.0  7.25  8.25  10.225  12.6   

        volatile acidity            ... sulphates       alcohol             \
                   count      mean  ...       75%   max   count       mean   
quality                             ...                                      
3                   10.0  0.884500  ...     0.615  0.86    10.0   9.955000   
4                   53.0  0.693962  ...     0.600  2.00    53.0  10.265094   
5                  681.0  0.577041  ...     0.660  1.98   681.0   9.899706   
6                  638.0  0.497484  ...     0.750  1.95   638.0  10.629519   
7                  199.0  0.403920  ...     0.830  1.36   199.0  11.465913   
8                   18.0  0.423333  ...     0.820  1.10    18.0  12.094444   

                                                      
              std  min     25%     50%     75%   max  
quality                                               
3        0.818009  8.4   9.725   9.925  10.575  11.0  
4        0.934776  9.0   9.600  10.000  11.000  13.1  
5        0.736521  8.5   9.400   9.700  10.200  14.9  
6        1.049639  8.4   9.800  10.500  11.300  14.0  
7        0.961933  9.2  10.800  11.500  12.100  14.0  
8        1.224011  9.8  11.325  12.150  12.875  14.0  

[6 rows x 88 columns]

In [27]:
# Ahora podemos aplicar nuestra propia función de agregación; esta función toma el valor máximo de la columna
# y resta el valor mínimo de la columna
vino.agg(max_a_min)

fixed acidity  volatile acidity  citric acid  residual sugar  \
quality                                                                 
3                  4.9             1.140         0.66             4.5   
4                  7.9             0.900         1.00            11.6   
5                 10.9             1.150         0.79            14.3   
6                  9.6             0.880         0.78            14.5   
7                 10.7             0.795         0.76             7.7   
8                  7.6             0.590         0.69             5.0   

         chlorides  free sulfur dioxide  total sulfur dioxide  density    pH  \
quality                                                                        
3            0.206                 31.0                  40.0  0.00609  0.47   
4            0.565                 38.0                 112.0  0.00760  1.16   
5            0.572                 65.0                 149.0  0.01059  0.86   
6            0.381                 71.0                 159.0  0.01362  1.15   
7            0.346                 51.0                 282.0  0.01256  0.86   
8            0.042                 39.0                  76.0  0.00800  0.84   

         sulphates  alcohol  
quality                      
3             0.46      2.6  
4             1.67      4.1  
5             1.61      6.4  
6             1.55      5.6  
7             0.97      4.8  
8             0.47      4.2

In [28]:
# También podemos pasar métodos de cadena a través del agregado
vino.agg('mean')

fixed acidity  volatile acidity  citric acid  residual sugar  \
quality                                                                 
3             8.360000          0.884500     0.171000        2.635000   
4             7.779245          0.693962     0.174151        2.694340   
5             8.167254          0.577041     0.243686        2.528855   
6             8.347179          0.497484     0.273824        2.477194   
7             8.872362          0.403920     0.375176        2.720603   
8             8.566667          0.423333     0.391111        2.577778   

         chlorides  free sulfur dioxide  total sulfur dioxide   density  \
quality                                                                   
3         0.122500            11.000000             24.900000  0.997464   
4         0.090679            12.264151             36.245283  0.996542   
5         0.092736            16.983847             56.513950  0.997104   
6         0.084956            15.711599             40.869906  0.996615   
7         0.076588            14.045226             35.020101  0.996104   
8         0.068444            13.277778             33.444444  0.995212   

               pH  sulphates    alcohol  
quality                                  
3        3.398000   0.570000   9.955000  
4        3.381509   0.596415  10.265094  
5        3.304949   0.620969   9.899706  
6        3.318072   0.675329  10.629519  
7        3.290754   0.741256  11.465913  
8        3.267222   0.767778  12.094444

In [30]:
# Agreguemos una proporción entre la calidad (quality) y el contenido alcoholico (alcohol)
dframe_vino['qual/alc ratio'] = dframe_vino['quality']/dframe_vino['alcohol']
dframe_vino.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  qual/alc ratio  
0      9.4        5        0.531915  
1      9.8        5        0.510204  
2      9.8        5        0.510204  
3      9.8        6        0.612245  
4      9.4        5        0.531915

In [31]:
# Tambien podemos usar pivot_table en lugar groupby

# Pivotear la calidad (quality) en la tabla
dframe_vino.pivot_table(index=['quality'])

alcohol  chlorides  citric acid   density  fixed acidity  \
quality                                                               
3         9.955000   0.122500     0.171000  0.997464       8.360000   
4        10.265094   0.090679     0.174151  0.996542       7.779245   
5         9.899706   0.092736     0.243686  0.997104       8.167254   
6        10.629519   0.084956     0.273824  0.996615       8.347179   
7        11.465913   0.076588     0.375176  0.996104       8.872362   
8        12.094444   0.068444     0.391111  0.995212       8.566667   

         free sulfur dioxide        pH  qual/alc ratio  residual sugar  \
quality                                                                  
3                  11.000000  3.398000        0.303286        2.635000   
4                  12.264151  3.381509        0.392724        2.694340   
5                  16.983847  3.304949        0.507573        2.528855   
6                  15.711599  3.318072        0.569801        2.477194   
7                  14.045226  3.290754        0.614855        2.720603   
8                  13.277778  3.267222        0.668146        2.577778   

         sulphates  total sulfur dioxide  volatile acidity  
quality                                                     
3         0.570000             24.900000          0.884500  
4         0.596415             36.245283          0.693962  
5         0.620969             56.513950          0.577041  
6         0.675329             40.869906          0.497484  
7         0.741256             35.020101          0.403920  
8         0.767778             33.444444          0.423333

## División, aplicación y combinación

In [32]:
# ¿Qué pasaría si quisiéramos saber el contenido de alcohol mas alto para cada rango de calidad?
# Podemos usar la mecánica de groupby para dividir-aplicar-combinar
# Se crea una función que asigne un rango a cada vino, según el contenido de alcohol, siendo 1 el mayor contenido de alcohol.
def rango(df):
    df['alc_content_rank'] = np.arange(len(df)) + 1
    return df

In [34]:
# Se ordena el dataframe por su valor de alcohol, en orden ascendente
dframe_vino.sort_values('alcohol',ascending=False,inplace=True)

# Se agrupa por su calidad y se aplica la función de rango
dframe_vino = dframe_vino.groupby('quality').apply(rango)
dframe_vino.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
652            15.9              0.36         0.65             7.5      0.096   
588             5.0              0.42         0.24             2.0      0.060   
142             5.2              0.34         0.00             1.8      0.050   
144             5.2              0.34         0.00             1.8      0.050   
1270            5.0              0.38         0.01             1.6      0.048   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
652                  22.0                  71.0  0.99760  2.98       0.84   
588                  19.0                  50.0  0.99170  3.72       0.74   
142                  27.0                  63.0  0.99160  3.68       0.79   
144                  27.0                  63.0  0.99160  3.68       0.79   
1270                 26.0                  60.0  0.99084  3.70       0.75   

      alcohol  quality  qual/alc ratio  alc_content_rank  
652      14.9        5        0.335570                 1  
588      14.0        8        0.571429                 1  
142      14.0        6        0.428571                 1  
144      14.0        6        0.428571                 2  
1270     14.0        6        0.428571                 3

In [36]:
# Ahora finalmente podemos llamar al dataframe donde elrango del contenido de alcohol (alc_content_rank) es 1
# Se obtiene el número de conteos de calidad
num_de_calidad = dframe_vino['quality'].value_counts()
num_de_calidad

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [37]:
# ¡Ahora mostraremos la información combinada, de los vinos que tienen el mayor contenido de alcohol para su rango!
dframe_vino[dframe_vino.alc_content_rank == 1].head(len(num_de_calidad))

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
652           15.9              0.36         0.65             7.5      0.096   
588            5.0              0.42         0.24             2.0      0.060   
142            5.2              0.34         0.00             1.8      0.050   
821            4.9              0.42         0.00             2.1      0.048   
45             4.6              0.52         0.15             2.1      0.054   
899            8.3              1.02         0.02             3.4      0.084   

     free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
652                 22.0                  71.0  0.99760  2.98       0.84   
588                 19.0                  50.0  0.99170  3.72       0.74   
142                 27.0                  63.0  0.99160  3.68       0.79   
821                 16.0                  42.0  0.99154  3.71       0.74   
45                   8.0                  65.0  0.99340  3.90       0.56   
899                  6.0                  11.0  0.99892  3.48       0.49   

     alcohol  quality  qual/alc ratio  alc_content_rank  
652     14.9        5        0.335570                 1  
588     14.0        8        0.571429                 1  
142     14.0        6        0.428571                 1  
821     14.0        7        0.500000                 1  
45      13.1        4        0.305344                 1  
899     11.0        3        0.272727                 1

## Tabulación cruzada

In [38]:
# Las tabulaciones cruzadas son, básicamente, un caso especial de tablas dinámicas
# Vamos a crear un conjunto de datos pequeño

datos ="""\
Muestra   Animal   Inteligencia
1        Perro     Listo
2 Perro Listo
3 Gato Tonto
4 Gato Tonto
5 Perro Tonto
6 Gato Listo"""

# Se almacena como un dataframe
dframe = pd.read_table(StringIO(datos),sep='\s+')
dframe

Muestra Animal Inteligencia
0        1  Perro        Listo
1        2  Perro        Listo
2        3   Gato        Tonto
3        4   Gato        Tonto
4        5  Perro        Tonto
5        6   Gato        Listo

In [39]:
# Ahora podemos crear una tabla de tabulación cruzada, que básicamente es solo una tabla de frecuencias
pd.crosstab(dframe.Animal,dframe.Inteligencia,margins=True)

Inteligencia  Listo  Tonto  All
Animal                         
Gato              1      2    3
Perro             2      1    3
All               3      3    6

## ¡¡¡Felicidades Hemos terminado con el procesamiento de datos!!!

[**Siguiente Lección**](Lecci%C3%B3n%2009%20-%20Visualizaci%C3%B3n_Datos.ipynb)